In [1]:
import json
import re

In [2]:
from unidecode import unidecode
from num2words import num2words
from thefuzz import fuzz

In [49]:
import pandas as pd

In [3]:
extracted_steno_path = 'sample_data/pdf_extractions/seg_steno_86_21_decembrie_2022.json' 
transcription_path = 'sample_data/transcriptions/21_Decembrie_202200001.json'

In [4]:
with open(extracted_steno_path, 'r') as f:
    sample_steno = json.load(f)

with open(transcription_path, 'r') as f:
    sample_trans = json.load(f)

In [5]:
# from nltk.tokenize import sent_tokenize
# from nltk import download
# from unidecode import unidecode

In [6]:
# download('punkt')

In [7]:
sample_trans = sample_trans['segments']

In [8]:
sample_stenos = {
    idx: content for idx, content in sample_steno.items() if re.search('Domnul|Doamna', content['start_text'])
}

In [9]:
sample_stenos = { str(idx): sample_stenos[str(idx)] for idx in range(6,10) }

In [10]:
sample_stenos

{'6': {'start_text': 'Domnul Virgil Guran:',
  'start_index': 6308,
  'end_text': 'Domnul Ștefan-Radu Oprea:',
  'end_index': 8029,
  'text': ' \nDoamnelor și domnilor senatori, \nDeclar deschisă ședința plenului Senatului de astăzi, 21 decembrie 2022. \nȘedința este condusă de Virgil Guran, vicepreședinte al Senatului, asistat de domnul senator \nSorin Lavric și domnul senator Ion-Narcis Mircescu, secretari ai Senatului. \nOrdinea de zi pentru ședința plenului de astăzi și programul de lucru, stabilite de  Biroul \npermanent al Senatului și Comitetul liderilor grupurilor parlamentare, au fost distribuite și afișate pe \npagina de internet a Senatului. \nDoamnelor și domnilor senatori, \nPotrivit  dispozițiilor art.137, coroborat  cu prevederile art.136 din  Regulamentul Senatului, \nconform hotărârii Biroului permanent al Senatului, adoptată cu acordul prealabil al Comitetului liderilor \ngrupurilor  parlamentare,  ședința  plenului  Senatului  se  desfășoară  prin  mijloace  electron

In [11]:
def text_cleaning(text):
    
    text = text.lower()
    text = text.replace("art.", "art ")
    
    text = unidecode(text)

    # replace spaces
    text = re.sub("\s\s+", ' ', text)

    # parenthesis
    text = re.sub("\(.+?\)", ' ', text)

    # page number
    text = re.sub("-\s[0-9]+\s-", '', text)

    # special characters
    text = re.sub("[^\w\s]", ' ', text)
    
    # replace spaces
    text = re.sub("\s\s+", ' ', text)
    
    text = text.split(" ")
    text = [num2words(word, lang = 'ro') if word.isnumeric() else word for word in text]
    
    text = " ".join(text)
    
    text = unidecode(text)

    text = text.strip()
    
    return text


#### Various

In [12]:
f_seg = [segment_dict['text'] for segment_dict in sample_trans['segments'] if segment_dict['no_speech_prob'] < .95]

TypeError: list indices must be integers or slices, not str

In [ ]:
f_seg = [text_cleaning(seg) for seg in f_seg]

In [ ]:
A = f_seg[:50]

In [ ]:
A

[' la revedere ',
 ' la revedere ',
 ' doamnelor si domnilor senatori ',
 ' declar deschis sa sedinta plenului senatului de astazi douazeci si unu decembrie doua mii douazeci si doi ',
 ' sedinta este condusa de virgil guran vicepresedinte al senatului asistat de domnul senator sorin lavric si domnul senator ion narcis mircescu secretar ai senatului ',
 ' ordinea de zi pentru sedinta plenului de astazi si programul de lucru stabilite de biroul permanent al senatului si comitetul liderilor grupurilor parlamentare au fost distribuite si afisate pe pagina de internet al senatului ',
 ' doamnelor si domnilor senatori potrivit dispozitiilor al art o suta treizeci si sapte corroborat cu prevederile al art o suta treizeci si sase din regulamentul senatului ',
 ' conform hotararii biroului permanent al senatului adoptata cu acordul pe alabii la comitetul liderilor grupurilor parlamentare sedinta plenului senatului se desfasoara prin mijloace electronice in conformitate cu procedura prevazuta l

In [ ]:
steno_segment_0 = sample_steno['7']['text']
steno_segment_0 = text_cleaning(steno_segment_0)
steno_segment_0

' multumesc mult domnule presedinte de sedinta doresc sa am o scurta interventie pentru ca astazi este douazeci si unu decembrie si pentru ca in urma cu treizeci si trei de ani in aceasta zi dictatorul incerca sa mai ramana la conducerea tarii convocand o mare adunare in bucuresti foarte aproape de aceasta locatie care apoi s a transformat intr o revolutie desi ea a inceput de la timisoara v as ruga pentru toti aceia care s au jertfit in acea perioada pentru ca noi astazi sa putem sa traim intr o tara democratica si v as ruga de asemenea sa avem si acel moment de introspectie de reflectie daca ceea ce si au dorit atunci cei care s au jertfit am realizat in deplinatate astazi dar aceasta este o chestiune a dumneavoastra a doamnelor si domnilor senatori sa stiti daca facem lucrurile asa cum trebuie si asa cum si ar fi dorit ei dar pentru ei va rog sa tinem un moment de reculegere va multumesc '

In [ ]:
len(steno_segment_0)

903

In [ ]:
A

[' la revedere ',
 ' la revedere ',
 ' doamnelor si domnilor senatori ',
 ' declar deschis sa sedinta plenului senatului de astazi douazeci si unu decembrie doua mii douazeci si doi ',
 ' sedinta este condusa de virgil guran vicepresedinte al senatului asistat de domnul senator sorin lavric si domnul senator ion narcis mircescu secretar ai senatului ',
 ' ordinea de zi pentru sedinta plenului de astazi si programul de lucru stabilite de biroul permanent al senatului si comitetul liderilor grupurilor parlamentare au fost distribuite si afisate pe pagina de internet al senatului ',
 ' doamnelor si domnilor senatori potrivit dispozitiilor al art o suta treizeci si sapte corroborat cu prevederile al art o suta treizeci si sase din regulamentul senatului ',
 ' conform hotararii biroului permanent al senatului adoptata cu acordul pe alabii la comitetul liderilor grupurilor parlamentare sedinta plenului senatului se desfasoara prin mijloace electronice in conformitate cu procedura prevazuta l

In [ ]:
seg_sentences = sent_tokenize(steno_segment_0)
seg_sentences

[' multumesc mult domnule presedinte de sedinta doresc sa am o scurta interventie pentru ca astazi este douazeci si unu decembrie si pentru ca in urma cu treizeci si trei de ani in aceasta zi dictatorul incerca sa mai ramana la conducerea tarii convocand o mare adunare in bucuresti foarte aproape de aceasta locatie care apoi s a transformat intr o revolutie desi ea a inceput de la timisoara v as ruga pentru toti aceia care s au jertfit in acea perioada pentru ca noi astazi sa putem sa traim intr o tara democratica si v as ruga de asemenea sa avem si acel moment de introspectie de reflectie daca ceea ce si au dorit atunci cei care s au jertfit am realizat in deplinatate astazi dar aceasta este o chestiune a dumneavoastra a doamnelor si domnilor senatori sa stiti daca facem lucrurile asa cum trebuie si asa cum si ar fi dorit ei dar pentru ei va rog sa tinem un moment de reculegere va multumesc']

In [ ]:
fuzz.ratio(A[2],steno_segment_0)

7

In [ ]:
collector = []
for transcript_segment in A:
    match_score = fuzz.token_set_ratio(transcript_segment, steno_segment_0)
    print( match_score, transcript_segment)

31  la revedere 
31  la revedere 
100  doamnelor si domnilor senatori 
66  declar deschis sa sedinta plenului senatului de astazi douazeci si unu decembrie doua mii douazeci si doi 
29  sedinta este condusa de virgil guran vicepresedinte al senatului asistat de domnul senator sorin lavric si domnul senator ion narcis mircescu secretar ai senatului 
36  ordinea de zi pentru sedinta plenului de astazi si programul de lucru stabilite de biroul permanent al senatului si comitetul liderilor grupurilor parlamentare au fost distribuite si afisate pe pagina de internet al senatului 
48  doamnelor si domnilor senatori potrivit dispozitiilor al art o suta treizeci si sapte corroborat cu prevederile al art o suta treizeci si sase din regulamentul senatului 
38  conform hotararii biroului permanent al senatului adoptata cu acordul pe alabii la comitetul liderilor grupurilor parlamentare sedinta plenului senatului se desfasoara prin mijloace electronice in conformitate cu procedura prevazuta la art

In [ ]:
collector = []
for transcript_segment in A:
    match_score = fuzz.partial_ratio(transcript_segment, steno_segment_0)
    print( match_score, transcript_segment)

46  la revedere 
46  la revedere 
53  doamnelor si domnilor senatori 
62  declar deschis sa sedinta plenului senatului de astazi douazeci si unu decembrie doua mii douazeci si doi 
52  sedinta este condusa de virgil guran vicepresedinte al senatului asistat de domnul senator sorin lavric si domnul senator ion narcis mircescu secretar ai senatului 
48  ordinea de zi pentru sedinta plenului de astazi si programul de lucru stabilite de biroul permanent al senatului si comitetul liderilor grupurilor parlamentare au fost distribuite si afisate pe pagina de internet al senatului 
56  doamnelor si domnilor senatori potrivit dispozitiilor al art o suta treizeci si sapte corroborat cu prevederile al art o suta treizeci si sase din regulamentul senatului 
46  conform hotararii biroului permanent al senatului adoptata cu acordul pe alabii la comitetul liderilor grupurilor parlamentare sedinta plenului senatului se desfasoara prin mijloace electronice in conformitate cu procedura prevazuta la art 

In [ ]:
print(f"\n\n{A[5]}\n\n")
for idx, p_seg in enumerate(seg_sentences):
        print(fuzz.partial_ratio(p_seg, A[5]))
        print(idx, p_seg)
        print()



ordinea de zi pentru sedinta plenului de astazi si programul de lucru stabilite de biroul permanent al senatului si comitetul liderilor grupurilor parlamentare au fost distribuite si afisate pe pagina de internet al senatului.


50
0 multumesc mult, domnule presedinte de sedinta.

47
1 doresc sa am o scurta interventie.

46
2 pentru ca astazi este 21 decembrie si pentru ca, in urma cu 33 de ani, in aceasta zi, dictatorul incerca sa mai ramana la conducerea tarii convocand o mare adunare in bucuresti, foarte aproape de aceasta locatie, care apoi s-a transformat intr-o revolutie, desi ea a inceput de la timisoara, v-as ruga, pentru toti aceia care s-au jertfit in acea perioada pentru ca noi, astazi, sa putem sa traim intr-o tara democratica.

46
3 si v-as ruga, de asemenea, sa avem si acel moment de introspectie, de reflectie, daca ceea ce si-au dorit atunci cei care s-au jertfit am realizat in deplinatate astazi  dar aceasta este o chestiune a dumneavoastra, a doamnelor si domnilor se

#### Cont

In [13]:
# Text cleaning 
for steno_idx, steno_attrs in sample_stenos.items():
    steno_attrs['text'] = text_cleaning(steno_attrs['text'])

for transcript_seg in sample_trans:
    transcript_seg['text'] = text_cleaning(transcript_seg['text'])

In [19]:
sample_trans_ = {
    segment_transcript['id']: {
       key: value for key, value in segment_transcript.items() if key != 'id'
    }
    for segment_transcript in sample_trans}

In [27]:
sample_stenos_ = {
    int(idx): steno_value for idx, (steno_key, steno_value) in enumerate(sample_stenos.items())
}

In [28]:
sample_trans_

{0: {'seek': 12000,
  'start': 120.0,
  'end': 122.0,
  'text': 'la revedere',
  'tokens': [50364, 2369, 319, 937, 323, 0, 50464],
  'temperature': 0.4,
  'avg_logprob': -0.9817119836807251,
  'compression_ratio': 0.6,
  'no_speech_prob': 0.9564675688743591},
 1: {'seek': 15000,
  'start': 150.0,
  'end': 152.0,
  'text': 'la revedere',
  'tokens': [50364, 2369, 319, 937, 323, 0, 50464],
  'temperature': 0.0,
  'avg_logprob': -0.6423652768135071,
  'compression_ratio': 0.6,
  'no_speech_prob': 0.9613750576972961},
 2: {'seek': 18000,
  'start': 180.0,
  'end': 182.0,
  'text': 'la revedere',
  'tokens': [50364, 2369, 319, 937, 323, 0, 50464],
  'temperature': 0.0,
  'avg_logprob': -0.3844093084335327,
  'compression_ratio': 0.6,
  'no_speech_prob': 0.9724004864692688},
 3: {'seek': 21000,
  'start': 210.0,
  'end': 212.0,
  'text': 'la revedere',
  'tokens': [50364, 2369, 319, 937, 323, 0, 50464],
  'temperature': 0.0,
  'avg_logprob': -0.30875900387763977,
  'compression_ratio': 0.6,


In [29]:
sample_stenos_

{0: {'start_text': 'Domnul Virgil Guran:',
  'start_index': 6308,
  'end_text': 'Domnul Ștefan-Radu Oprea:',
  'end_index': 8029,
  'text': 'doamnelor si domnilor senatori declar deschisa sedinta plenului senatului de astazi douazeci si unu decembrie doua mii douazeci si doi sedinta este condusa de virgil guran vicepresedinte al senatului asistat de domnul senator sorin lavric si domnul senator ion narcis mircescu secretari ai senatului ordinea de zi pentru sedinta plenului de astazi si programul de lucru stabilite de biroul permanent al senatului si comitetul liderilor grupurilor parlamentare au fost distribuite si afisate pe pagina de internet a senatului doamnelor si domnilor senatori potrivit dispozitiilor art o suta treizeci si sapte coroborat cu prevederile art o suta treizeci si sase din regulamentul senatului conform hotararii biroului permanent al senatului adoptata cu acordul prealabil al comitetului liderilor grupurilor parlamentare sedinta plenului senatului se desfasoara p

In [16]:
FUZZY_THRESHOLD = 80

In [31]:
def fuzzy_search(transcription_segment, steno_segment):
    
    value = fuzz.token_set_ratio(transcription_segment, steno_segment)
    
    if value >= FUZZY_THRESHOLD:
        return True
    else: 
        return False
    

In [62]:
len(sample_trans_)

821

In [68]:
# %%
running_transcript_segments = [segment for segment in sample_trans_.values()]
running_steno_segments = sample_stenos_

forward_limit = 20

associated_segments = {}


transcript_idx = 0

for steno_segment_idx, steno_segment in list(sample_stenos_.items())[:5]:
    
    found_match = False

    current_steno_text = steno_segment['text']
    running_transcript_len = len(running_transcript_segments)
    
    # Find first match 
    while(transcript_idx < running_transcript_len and not found_match):
        
        current_transcript_text = running_transcript_segments[transcript_idx]['text']

        # Assign first match
        if fuzzy_search(current_transcript_text, current_steno_text):
            if steno_segment_idx not in associated_segments:
                associated_segments[steno_segment_idx] = {
                    'steno_text': current_steno_text,
                    'transcript_text': current_transcript_text,
                }
            found_match = True
                
        transcript_idx += 1

    # As it has been assigned, modify index to the next one:
    # transcript_idx += 1
    
    rolling_count = 1
    consecutive = True
    while(
        (transcript_idx < (transcript_idx + forward_limit)) and \
        (transcript_idx < running_transcript_len) and consecutive):

        current_transcript_text = running_transcript_segments[transcript_idx]['text']
        if fuzzy_search(current_transcript_text, current_steno_text):
            associated_segments[steno_segment_idx]['transcript_text'] = \
                associated_segments[steno_segment_idx]['transcript_text'] + " " \
                + current_transcript_text
            
            current_len = len(associated_segments[steno_segment_idx]['transcript_text'])

            transcript_idx += 1
        else:
            consecutive = False
        
        
    

In [69]:
associated_segments

{0: {'steno_text': 'doamnelor si domnilor senatori declar deschisa sedinta plenului senatului de astazi douazeci si unu decembrie doua mii douazeci si doi sedinta este condusa de virgil guran vicepresedinte al senatului asistat de domnul senator sorin lavric si domnul senator ion narcis mircescu secretari ai senatului ordinea de zi pentru sedinta plenului de astazi si programul de lucru stabilite de biroul permanent al senatului si comitetul liderilor grupurilor parlamentare au fost distribuite si afisate pe pagina de internet a senatului doamnelor si domnilor senatori potrivit dispozitiilor art o suta treizeci si sapte coroborat cu prevederile art o suta treizeci si sase din regulamentul senatului conform hotararii biroului permanent al senatului adoptata cu acordul prealabil al comitetului liderilor grupurilor parlamentare sedinta plenului senatului se desfasoara prin mijloace electronice in conformitate cu procedura prevazuta la art o suta unsprezece din regulamentul senatului in ca